## [Behavioral] Interpreter Method

![Interpreter Method](https://www.planttext.com/plantuml/png/pLEzQiCm4Du5UeSRRKgWiwUKqg53Iw4F46JdceZOaj6T8At9krVoJweObmIwLAE-tpkaDR5ox9jQ2Zfeqogd6gXgHVJ0x7Jf6PyhQzxr5yAz55Bq63nQmtXawAI0S5Qm2LhFK02nEzzLtZaql6ajQTRM15WR7k0xs24FgYnFTGCmb_QSH4nBdCkSVCcFxG4JirCSIPKXN5KCJyVM8L5qw2PnDw6dg40N69nLu_Z5j8XPkqJKkd2VMSh7i65_hxufqILvmGFDbXOgX3sQddqPSNkPvGPljQTRyslyYFO_s1bnUhTVWlxUxqMR_rfmjZgxMgMj2eXjhhFwPbBCv_p5xIfjj9DYhxS8sBJy2Gt6TqDIhCFg-Z_-3G00)

In [8]:
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import Optional

In [9]:
class Context:
    """문맥: 현재 환경 정보를 저장"""
    def __init__(self, input_str: str):
        self._input = input_str
        self._current_position = 0

    def get_input(self) -> str:
        return self._input

    def get_current_position(self) -> int:
        return self._current_position

    def set_current_position(self, position: int) -> None:
        self._current_position = position

    def get_current_token(self) -> int:
        if self._current_position < len(self._input):
            return int(self._input[self._current_position])
        return -1

class Expression(ABC):
    """표현식 인터페이스"""
    @abstractmethod
    def interpret(self, context: Context) -> int:
        """해석"""
        pass

    @abstractmethod
    def print(self, os) -> None:
        """출력"""
        pass

class NumberExpression(Expression):
    """숫자 표현식"""
    def __init__(self, number: int):
        self._number = number

    def interpret(self, context: Context) -> int:
        return self._number

    def print(self, os) -> None:
        os.write(str(self._number))

class PlusExpression(Expression):
    """덧셈 표현식"""
    def __init__(self, left: Expression, right: Expression):
        self._left = left
        self._right = right

    def interpret(self, context: Context) -> int:
        return self._left.interpret(context) + self._right.interpret(context)

    def print(self, os) -> None:
        os.write("(")
        self._left.print(os)
        os.write(" + ")
        self._right.print(os)
        os.write(")")

class MinusExpression(Expression):
    """뺄셈 표현식"""
    def __init__(self, left: Expression, right: Expression):
        self._left = left
        self._right = right

    def interpret(self, context: Context) -> int:
        return self._left.interpret(context) - self._right.interpret(context)

    def print(self, os) -> None:
        os.write("(")
        self._left.print(os)
        os.write(" - ")
        self._right.print(os)
        os.write(")")

In [10]:
def parse(context: Context) -> Expression:
    """구문 분석"""
    current_token = context.get_current_token()
    context.set_current_position(context.get_current_position() + 1)
    left = NumberExpression(current_token)

    if context.get_current_position() < len(context.get_input()):
        op = context.get_input()[context.get_current_position()]
        context.set_current_position(context.get_current_position() + 1)
        right = parse(context)
        if op == '+':
            return PlusExpression(left, right)
        elif op == '-':
            return MinusExpression(left, right)
        else:
            raise ValueError("Invalid operator")

    return left

In [11]:
"""클라이언트 코드"""
import sys

expression_str = "1+2-3+4"
context = Context(expression_str)
try:
    expression = parse(context)

    print("입력된 수식: ", end="")
    expression.print(sys.stdout)
    print()

    result = expression.interpret(context)
    print(f"결과: {result}")
except ValueError as e:
    print(f"Error: {e}")

입력된 수식: (1 + (2 - (3 + 4)))
결과: -4


### Plant UML

```plantuml
@startuml
skinparam classAttributeIconSize 0

class Context {
    - _input : str
    - _current_position : int
    + get_input() : str
    + get_current_position() : int
    + set_current_position(position : int)
    + get_current_token() : int
}

abstract Expression {
    + {abstract} interpret(context : Context) : int
    + {abstract} print(os)
}

class NumberExpression extends Expression {
    - _number : int
    + interpret(context : Context) : int
    + print(os)
}

class PlusExpression extends Expression {
    - _left : Expression
    - _right : Expression
    + interpret(context : Context) : int
    + print(os)
}

class MinusExpression extends Expression {
    - _left : Expression
    - _right : Expression
    + interpret(context : Context) : int
    + print(os)
}

PlusExpression o-- Expression : left
PlusExpression o-- Expression : right
MinusExpression o-- Expression : left
MinusExpression o-- Expression : right

hide empty members
@enduml
```